In [2]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [3]:
url="http://wblivesurf.com"
r=requests.get(url)
soup=BeautifulSoup(r.content,'html.parser')

In [4]:
print(soup.prettify())
#print(soup.get_text())

<!DOCTYPE doctype html>
<html>
 <head>
  <meta charset="utf-8"/>
  <title>
   Wrightsville Beach Surf Forecasts, Surf Reports &amp; Live Cams
  </title>
  <meta content="Wrightsville Beach Surf Forecasts, Surf Reports &amp; Live Cams" name="title"/>
  <meta content="Copyright © WB Live Surf 2019. All Rights Reserved." name="Copyright"/>
  <meta content="width=device-width, initial-scale=1.0 minimal-ui" name="viewport">
   <link href="http://www.wblivesurf.com/wp-content/themes/WBLiveSurf/img/favicon.ico" rel="shortcut icon"/>
   <link href="http://www.wblivesurf.com/wp-content/themes/WBLiveSurf/reset.css" rel="stylesheet"/>
   <link href="http://www.wblivesurf.com/wp-content/themes/WBLiveSurf/style.css" rel="stylesheet"/>
   <link href="http://www.wblivesurf.com/xmlrpc.php" rel="pingback"/>
   <!-- This site is optimized with the Yoast SEO plugin v12.5 - https://yoast.com/wordpress/plugins/seo/ -->
   <meta content="Watch and view detailed surf reports and weather forecasts from our li

In [5]:
conditions={}

for strong_tag in soup.find_all('strong'):
    
    conditions.update({strong_tag.text : strong_tag.next_sibling})

In [6]:
conditions

{'Today’s Surf': ' Report',
 'Current': ' Weather',
 'C-Street': ' / 12.17.19',
 'Sheldon At SweetWater Surf Shop': None,
 'No Logos': '\xa0= Flat or Really Bad. Pretty much unrideable! A good time to get other things done.',
 '1 Logo': ' = Rideable but not good form. Might not be worth the time or effort but there could be a few okay waves.',
 '2 Logos': '\xa0= Could be fun. Rideable and a few okay waves to be ridden.',
 '•3 Logos': '\xa0= Fun! Better than the average day. Worth paddling out depending on the surface conditions.',
 '4 Logos': '\xa0= Really fun. Paddle out if you can. One of the better days of the year.',
 '5 Logos': '\xa0= Rare Epic or All Time conditions. Quitting work or skipping school is an option but not encouraged. Why are you still reading the report! Paddle out now!',
 'Location:': ' Wrightsville Beach, NC',
 'Swell Size:': ' Knee-Thigh High',
 'Water Surface:': ' Clean',
 'Water Temp:': ' 56',
 'Wind:': ' SW 5-10 mph',
 'High Tide:': ' 11:06:00 AM',
 'Low Tide

In [7]:
author=list(conditions.keys())[3]
size=conditions['Swell Size:'].lstrip()
surface=conditions['Water Surface:'].lstrip()
temp=conditions['Water Temp:'].lstrip()
wind=conditions['Wind:'].lstrip()
high=conditions['High Tide:'].lstrip()
low=conditions['Low Tide:'].lstrip()
rise=conditions['Sunrise:'].lstrip()
sunset=conditions['Sunset:'].lstrip()

In [8]:
print(author)
print(size)
print(surface)
print(temp)
print(wind)


Sheldon At SweetWater Surf Shop
Knee-Thigh High
Clean
56
SW 5-10 mph


In [9]:
report=soup.find('div',attrs={'id':'wbMainReport'}).text

In [10]:
report

"\nC-Street / 12.17.19Afternoon Update3:34 PM / BY: Sheldon At SweetWater Surf ShopStill a little wave out there and its pretty clean now.\xa0 A few guys out there now but I didn't see them catch much while I was on the beach.\xa0 I would go surf....It's warm and there is a little wave!\xa0 Hope everyone had a great day.\r\n\t\t\t\t\t\t\t"

In [41]:
pattern=r"(\d..\d..\d.)"
date=re.findall(pattern,report)

In [42]:
pattern=r"\d([a-zA-z].+?)\d"
when=re.findall(pattern,report)

In [56]:
pattern=r"[a-z](\d*\:.....)"
time=re.findall(pattern,report)

In [45]:
pattern="([a-zA-Z].+?)\s.+"
author=re.findall(pattern,author)

In [46]:
pattern="[o][p](\w.+)"
text=re.findall(pattern,report)

In [47]:
print(date)
print(when)
print(time)
print(author)
print(text)

['11.24.19']
['Lunchtime Report']
['1:25 PM']
['Brandon']
["The waves definitely picked up a little bit since this morning. Conditions are pretty clean with little wind. Probably in the knee to waist high rang and should be fun on a bigger board. It's a beautiful day. Get out there!\r", 'e From Helen Surf Silent Auction – Check out all the great items up for grabs. \xa0Promote your business. \xa0Taking Silent Auction items now!']


In [57]:
date=date[0]
when=when[0]
time=time[0]
author=author[0]
text=text[0]

In [58]:
report_dict={'date':date,'part_of_day':when,'time':time,'author':author,'size':size,'surface':surface,
             'temp':temp,'wind':wind,'high':high,'low':low,'rise':rise,'sunset':sunset,'text':text}

In [53]:
report_dict

{'date': ['11.24.19'],
 'part_of_day': ['Lunchtime Report'],
 'time': ['1:25 PM'],
 'author': ['Brandon'],
 'size': 'Knee/Thigh High',
 'surface': 'Clean',
 'temp': '60.4',
 'wind': 'NW 5-10 mph',
 'high': '11:34:00 PM',
 'low': '11:19:00 AM',
 'rise': '6:52:00 AM',
 'sunset': '5:03:00 PM',
 'text': ["The waves definitely picked up a little bit since this morning. Conditions are pretty clean with little wind. Probably in the knee to waist high rang and should be fun on a bigger board. It's a beautiful day. Get out there!\r",
  'e From Helen Surf Silent Auction – Check out all the great items up for grabs. \xa0Promote your business. \xa0Taking Silent Auction items now!']}

In [59]:
report_df=pd.DataFrame(report_dict)

ValueError: If using all scalar values, you must pass an index

In [55]:
report_df

,date,part_of_day,time,author,size,surface,temp,wind,high,low,rise,sunset,text
0,11.23.19,Lunch-Time Update,12:36 PM,Dylan,Knee High,Light Bump,60,N 5-10 mph,4:21:00 PM,10:44:00 PM,6:51:00 AM,5:03:00 PM,Good Afternoon everyone! It's still looking qu...


In [56]:
report_df.to_csv(r'/home/pi/Documents/Surf project/reports')

In [57]:
report_df.to_csv(r'/home/pi/Documents/Surf project/reports',mode='a',header=False)